In [ ]:
from langchain_text_splitters import CharacterTextSplitter
import os
os.environ["OCR_AGENT"] = "pytesseract"  # 꼭 partition_pdf 임포트 전에
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.pdf import partition_pdf

In [ ]:
def extract_pdf_elements(path, fname):
    return partition_pdf(
        filename=path + fname,
        extract_images_in_pdf=True,  # PDF에서 이미지를 추출
        infer_table_structure=True,  # 테이블 구조를 추론
        chunking_strategy="by_title",  # 타이틀을 기준으로 텍스트를 블록으로 분할
        max_characters=4000,  # 최대 4000자로 텍스트 블록을 제한
        new_after_n_chars=3800,  # 3800자 이후에 새로운 블록 생성
        combine_text_under_n_chars=2000,  # 2000자 이하의 텍스트는 결합
        image_output_dir_path=path,  # 이미지가 저장될 경로 설정
   languages=['kor']
        # image_output_dir_path=os.path.join(os.getcwd(),"figures"),
    )
raw_data = extract_pdf_elements("./data/", "RRF.pdf")
def categorize_elements(raw_pdf_elements):
    """
    PDF에서 추출한 요소들을 테이블과 텍스트로 분류합니다.
    raw_pdf_elements: unstructured.documents.elements 리스트
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))  # 테이블 요소를 저장
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))  # 텍스트 요소를 저장
    return texts, tables

texts, tables = categorize_elements(raw_data)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=200
)
joined_texts = " ".join(texts)
text_token = text_splitter.split_text(joined_texts)